In [1]:
import pandas as pd
# import hvplot.pandas
import re
# from pycaret.classification import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf


In [2]:
file_path1 = "wine_data.csv"
df= pd.read_csv(file_path1)

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,wine_type,winery
0,42302,Turkey,"Apricot blush in color, this wine has aromas o...",Seahorse Rosé,86,19.0,Aegean,NaN,NaN,Mike DeSimone,@worldwineguys,Pasaeli 2015 Seahorse Rosé Çalkarası (Aegean),Çalkarası,white,Pasaeli
1,66086,Turkey,"Deep-salmon to the eye, this wine has a nose o...",Mermaid Rosé,87,19.0,Aegean,NaN,NaN,Mike DeSimone,@worldwineguys,Pasaeli 2015 Mermaid Rosé Çalkarası (Aegean),Çalkarası,white,Pasaeli
2,51421,Macedonia,This Macedonian wine made from 100% Žilavka ha...,NaN,88,15.0,Tikves,NaN,NaN,Jeff Jenssen,@worldwineguys,Stobi 2014 Žilavka (Tikves),Žilavka,white,Stobi
3,45401,US,"Comprised 100% of this rare variety, this wine...",Moonlight Sonata,85,75.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Cerridwen 2012 Moonlight Sonata Abouriou (Russ...,Abouriou,red,Cerridwen
4,98235,France,Abouriou is a grape found almost exclusively i...,Just,87,15.0,Southwest France,Côtes du Marmandais,NaN,Roger Voss,@vossroger,Cave du Marmandais 2012 Just Abouriou (Côtes d...,Abouriou,red,Cave du Marmandais


In [3]:
# import sweetviz as sv

# my_report = sv.analyze(df)
# my_report.show_html()

In [4]:
df.isna().sum()

Unnamed: 0                   0
country                     56
description                  0
designation              37358
points                       0
price                     8973
province                    56
region_1                 21112
region_2                 79160
taster_name              26178
taster_twitter_handle    31127
title                        0
variety                      0
wine_type                    0
winery                       0
dtype: int64

In [5]:
df.count()

Unnamed: 0               129626
country                  129570
description              129626
designation               92268
points                   129626
price                    120653
province                 129570
region_1                 108514
region_2                  50466
taster_name              103448
taster_twitter_handle     98499
title                    129626
variety                  129626
wine_type                129626
winery                   129626
dtype: int64

In [6]:
# removing any lines without target values
df = df[df['price'].notna()]

In [7]:
# confirming na rows were deleted
df.count()

Unnamed: 0               120653
country                  120601
description              120653
designation               85979
points                   120653
price                    120653
province                 120601
region_1                 101200
region_2                  50247
taster_name               96219
taster_twitter_handle     91318
title                    120653
variety                  120653
wine_type                120653
winery                   120653
dtype: int64

In [8]:
df.province.value_counts()

California           36078
Washington            8576
Oregon                5357
Tuscany               5124
Bordeaux              4001
                     ...  
Dealurile Hușilor        1
Kathikas                 1
Dalmatian Coast          1
Commandaria              1
Imathia                  1
Name: province, Length: 415, dtype: int64

In [9]:
df = df.drop(labels='Unnamed: 0', axis=1)


In [10]:
df.describe()

,points,price
count,120653.000000,120653.000000
mean,88.423728,35.381897
std,3.045260,41.038540
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


In [11]:
# df.to_csv('cleaned_wine_data.csv', index=False)

In [12]:
province = df.province.value_counts()

# province.hvplot.bar()

In [13]:
province.head(20)

California            36078
Washington             8576
Oregon                 5357
Tuscany                5124
Bordeaux               4001
Northern Spain         3793
Mendoza Province       3226
Burgundy               3105
Piedmont               3086
New York               2666
Veneto                 2314
Alsace                 2146
Northeastern Italy     1959
Loire Valley           1591
Sicily & Sardinia      1548
Champagne              1383
Southwest France       1332
South Australia        1331
Catalonia              1151
Provence               1135
Name: province, dtype: int64

In [14]:
# p_df = df.copy()
# p_df = p_df.groupby('province')
# p_df.head()

In [15]:
top_ten_list = ['California','Washington','Bordeaux','Tuscany','Oregon','Burgundy','Northern Spain','Piedmont','Mendoza Province','Veneto']

top_ten_provinces = df.query('province in @top_ten_list')


top_ten_df = pd.DataFrame(top_ten_provinces)
top_ten_df.count()

country                  74660
description              74660
designation              50044
points                   74660
price                    74660
province                 74660
region_1                 74582
region_2                 47581
taster_name              54402
taster_twitter_handle    54402
title                    74660
variety                  74660
wine_type                74660
winery                   74660
dtype: int64

In [16]:
top_ten_df['points'].isna().sum()

0

In [17]:
df_desc_desig = top_ten_df[['wine_type','designation','description']].copy()


In [18]:
# df_desc_desig = df_desc_desig[df_desc_province['designation'].notna()]
# df_desc_desig.count()

In [19]:
# df_desc_desig.to_csv("desc_designation_data_CLEAN.csv", index=False)

In [20]:
df_prov_price = top_ten_df[['province','wine_type']]
df_prov = top_ten_df[['price','points']]


In [21]:
# Generate our categorical variable lists
wine_cat = df_prov_price.dtypes[df_prov_price.dtypes == "object"].index.tolist()

In [22]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_prov_price[wine_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(wine_cat)
encode_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,province_Bordeaux,province_Burgundy,province_California,province_Mendoza Province,province_Northern Spain,province_Oregon,province_Piedmont,province_Tuscany,province_Veneto,province_Washington,wine_type_other,wine_type_red,wine_type_white
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
df_prov.head()

,price,points
3,75.0,85
91,30.0,91
112,28.0,89
178,30.0,91
234,25.0,87


In [24]:
# Merge one-hot encoded features and drop the originals
df_merge = df_prov.merge(encode_df,left_index=True, right_index=True)
# df_prov = df_prov.drop(wine_cat,1)
df_merge.head()

# df_concat = pd.concat([df_prov, encode_df], axis=1)


,price,points,province_Bordeaux,province_Burgundy,province_California,province_Mendoza Province,province_Northern Spain,province_Oregon,province_Piedmont,province_Tuscany,province_Veneto,province_Washington,wine_type_other,wine_type_red,wine_type_white
3,75.0,85,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
91,30.0,91,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
112,28.0,89,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
178,30.0,91,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
234,25.0,87,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


0

In [25]:
# Split our preprocessed data into our features and target arrays
y = df_merge["wine_type_red"].values
X = df_merge.drop(["wine_type_red","wine_type_white","wine_type_other"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-25-8a081e35fda7>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df_merge.drop(["wine_type_red","wine_type_white","wine_type_other"],1).values


In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [79]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 8)                 104       
                                                                 
 dense_49 (Dense)            (None, 5)                 45        
                                                                 
 dense_50 (Dense)            (None, 1)                 6         
                                                                 
Total params: 155
Trainable params: 155
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1114/1114 [==============================] - 4s 3ms/step - loss: 0.5642 - accuracy: 0.7413
Epoch 2/100
1114/1114 [==============================] - 3s 3ms/step - loss: 0.5437 - accuracy: 0.7447
Epoch 3/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5422 - accuracy: 0.7448
Epoch 4/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5413 - accuracy: 0.7451
Epoch 5/100
1114/1114 [==============================] - 5s 4ms/step - loss: 0.5407 - accuracy: 0.7453
Epoch 6/100
1114/1114 [==============================] - 3s 3ms/step - loss: 0.5403 - accuracy: 0.7453
Epoch 7/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5398 - accuracy: 0.7453
Epoch 8/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5394 - accuracy: 0.7462
Epoch 9/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5392 - accuracy: 0.7456
Epoch 10/100
1114/1114 [==============================] - 3s 3ms/step - l

In [82]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

372/372 - 1s - loss: 0.5573 - accuracy: 0.7295 - 662ms/epoch - 2ms/step
Loss: 0.5572815537452698, Accuracy: 0.7294543385505676


In [83]:
# Optimization attempt #1: add another layer, up the neurons
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 8)                 104       
                                                                 
 dense_52 (Dense)            (None, 8)                 72        
                                                                 
 dense_53 (Dense)            (None, 8)                 72        
                                                                 
 dense_54 (Dense)            (None, 1)                 9         
                                                                 
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________


In [84]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [85]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1114/1114 [==============================] - 6s 3ms/step - loss: 0.5675 - accuracy: 0.7308
Epoch 2/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7451
Epoch 3/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5444 - accuracy: 0.7447
Epoch 4/100
1114/1114 [==============================] - 3s 3ms/step - loss: 0.5434 - accuracy: 0.7456
Epoch 5/100
1114/1114 [==============================] - 3s 3ms/step - loss: 0.5430 - accuracy: 0.7454
Epoch 6/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5423 - accuracy: 0.7453
Epoch 7/100
1114/1114 [==============================] - 2s 2ms/step - loss: 0.5414 - accuracy: 0.7451
Epoch 8/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5409 - accuracy: 0.7455
Epoch 9/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5402 - accuracy: 0.7454
Epoch 10/100
1114/1114 [==============================] - 3s 3ms/step - l

In [86]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

372/372 - 1s - loss: 0.5566 - accuracy: 0.7301 - 829ms/epoch - 2ms/step
Loss: 0.556568443775177, Accuracy: 0.7301279902458191


In [96]:
# Optimization attempt #2: adding more neurons and include every type of activation method
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 12
# adding more layers and upping # of neurons
hidden_nodes_layer3 = 12
hidden_nodes_layer4 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="linear"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 12)                156       
                                                                 
 dense_71 (Dense)            (None, 12)                156       
                                                                 
 dense_72 (Dense)            (None, 12)                156       
                                                                 
 dense_73 (Dense)            (None, 12)                156       
                                                                 
 dense_74 (Dense)            (None, 1)                 13        
                                                                 
Total params: 637
Trainable params: 637
Non-trainable params: 0
_________________________________________________________________


In [97]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [98]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1114/1114 [==============================] - 5s 3ms/step - loss: 0.5723 - accuracy: 0.7194
Epoch 2/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5465 - accuracy: 0.7455
Epoch 3/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5432 - accuracy: 0.7457
Epoch 4/100
1114/1114 [==============================] - 4s 4ms/step - loss: 0.5421 - accuracy: 0.7461
Epoch 5/100
1114/1114 [==============================] - 5s 4ms/step - loss: 0.5410 - accuracy: 0.7461
Epoch 6/100
1114/1114 [==============================] - 3s 2ms/step - loss: 0.5408 - accuracy: 0.7458
Epoch 7/100
1114/1114 [==============================] - 5s 5ms/step - loss: 0.5402 - accuracy: 0.7461
Epoch 8/100
1114/1114 [==============================] - 5s 4ms/step - loss: 0.5394 - accuracy: 0.7459
Epoch 9/100
1114/1114 [==============================] - 4s 3ms/step - loss: 0.5392 - accuracy: 0.7458
Epoch 10/100
1114/1114 [==============================] - 3s 2ms/step - l

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [73]:
# Optimization attempts reveal no increased accuracy with additional layers/neurons/activation methods.
# This indicates that predicting wine types with only the province of origin is not an accurate estimator